In [ ]:
from vdorogu.inferencer.inference import Inferencer
from more_itertools import chunked
import torch

In [ ]:
from pathlib import Path
import os

In [ ]:
model_path = Path.home() / "IWeights" / "dense_retrieval_labse"

In [ ]:
inf = Inferencer(
    model='vdorogu/inferencer/models/web/labse_dense_retrieval_title.py',
    storage_path=None,
    model_data_path=str(model_path),
    batch_size=256,
    mode='document_emb',
    half=True,
    gpus=0,
    model_params={}
)

In [ ]:
from vdorogu.modeling.module import M1Model

In [ ]:
model = M1Model(model=inf)

In [ ]:
model.embedding_model

In [ ]:
from bertopic import BERTopic

In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
umap_model = UMAP(n_components=5, metric="cosine")

In [ ]:
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [ ]:
botpic_model = BERTopic(
    embedding_model=model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model
)

In [ ]:
import polars as pl

In [ ]:
path = Path.home() / "IDataset" / "vk" / "search_banner_phrases_2023-05-04.csv"

In [ ]:
df = pl.read_csv(str(path))

In [ ]:
df = df.sample(1000)

In [ ]:
docs = [str(x) for x in df.to_arrow()["search_phrase"]]

In [ ]:
docs[:10]

In [ ]:
embeddings = model.embed_documents(docs)

In [ ]:
assert embeddings.shape[0] == len(docs)

In [ ]:
botpic_model.fit_transform(docs)

In [ ]:
import numpy as np
import torch.nn as nn

In [ ]:
res = list(
    inf.inference_text_realtime(
            [
            "привет мир!",
            "hello world",
            "may the odds be ever in your favor"
            ]
        )
    )

In [ ]:
# Response for the x_i text sample: [i // batch_size] x [text_{i % batch_size}] x embedding

In [ ]:
res[0][1].shape

In [ ]:
np.sum(
    np.linalg.norm(
        list(inf.inference_text_realtime(["привет мир!"]))[0]
    )
)